## This script fits simple cross validated SVM using single fit and alpy

## It should be straightforward to go from this script to running experiments on any fingerprint/compound pair

In [ ]:
% load_ext autoreload
% autoreload 2
import matplotlib.pylab as plt
% matplotlib inline
import sys
import gzip
import seaborn
from os import path
sys.path.insert(0, "../mlls2015")
sys.path.insert(0, "..")
import training_data
from models.cv import AdaptiveGridSearchCV
from training_data.datasets import CVBaseChemDataset
import bunch
import numpy as np
from bunch import Bunch
import os
import logging
import json
from experiments.utils import wac_score
from misc.config import RESULTS_DIR
from misc.utils import config_log_to_file

import copy
from models.balanced_models import *
import cPickle
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.svm import SVC

from misc.utils import get_run_properties

import alpy
import alpy_addons
from alpy_addons.active import ActiveLearner
from alpy.datasets.generate import get_2d_gaussians
from alpy_addons.monitors import *
from alpy.oracle import SimulatedOracle
from six import iteritems

from alpy_addons.strategy import UncertaintySampling, PassiveStrategy
from alpy.utils import mask_unknowns

from sklearn.metrics import auc

from training_data.datasets import calculate_jaccard_kernel

from sklearn.grid_search import GridSearchCV

opts = Bunch({"C_min": -6, 
              "C_max": 5,
              "internal_cv": 4,
              "max_iter": 10000000,
              "n_folds": 5, 
              "preprocess": "clip01", #"max_abs", 
              "fold": 3, 
              "d": 1,
              "output_dir": "lol",
              "warm_start": 20, # TODO: add cluster-dependent warm_start
              "strategy_kwargs": "",
              "strategy": "UncertaintySampling",
              "compound": "5-HT1a", 
              "representation": "MACCS", 
              "jaccard": 1, 
              "rng": 777, 
              "name": "test_svm_al",
              "batch_size": 50})



In [ ]:
# TODO: write test that will:
# - check repeatibility of whole AL experiment
# - check that final accuracy is SAME as fitting just single estimator
# - check that we have a classifier that doesn't overfit!
# - sanity checks for results:
    # * CV not picked on the borders in more than 5% of cases
    # * performance is lower than 98% on labeled in 50% of time